In [11]:
import sys
sys.path.append('/home/grant/workdir/build/ssl-sky-surveys/')

import torch
import numpy as np
import matplotlib.pyplot as plt
import h5py
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

from utils.load_trained_model import load_model_from_checkpoint
%matplotlib inline
from astropy.visualization import make_lupton_rgb
import torchvision.transforms as transforms
import torchvision.models as models
import time

### Load model

In [5]:
ckpt = '/external/grant/models/moco/custom50jc/custom50jccheckpoint_0158.pth.tar'
model = load_model_from_checkpoint(ckpt,basenet=False,num_channels=3)

INFO:root:Loading checkpoint /external/grant/models/moco/custom50jc/custom50jccheckpoint_0158.pth.tar
INFO:root:Printing a pretrained model without FC layers
INFO:root:Chekpoint loaded. Checkpoint epoch 159


### Load data

In [7]:
#data = h5py.File('/external/grant/data/sdss_w_specz_valid.h5', 'r')
data = np.load('/external/grant/data/DES_imarray.npy')
data = data.reshape(data.shape[0],3,228,228)

In [8]:
#print("Data keys:", data.keys())
#print("Images shape:", data['images'].shape)

In [9]:

print(data.shape)
#imt = transforms.ToTensor()(img).unsqueeze_(0)

(5961, 3, 228, 228)


### Inference

In [12]:
t0 = time.time()
with torch.no_grad():
    #datai = torch.from_numpy(data[0:50])
    #datai = torch.from_numpy(data[0:20]) 
    #reps = model(datai).cpu().detach().numpy()

    datai = torch.from_numpy(np.flip(data[0:50],axis=(2,3)).copy())
    reps = model(datai.clone().detach()).cpu().detach().numpy()
    
print(time.time()-t0)

33.91937279701233


In [27]:
reps.shape

(50, 2048)

In [28]:
with torch.no_grad():
    for i in range(1,20):
        #datai = torch.from_numpy(data[50*i:50*(i+1)].copy())        
        datai = torch.from_numpy(np.flip(data[50*i:50*(i+1)],axis=(2,3)).copy())
        repsi = model(datai).cpu().detach().numpy()
        #repsi = model(torch.tensor(data['images'][200*i:200*(i+1)][:,:,::-1])).cpu().detach().numpy()
        reps = np.concatenate((reps,repsi))

In [29]:
reps.shape

(1000, 2048)

In [31]:
np.save('/external/grant/models/moco/custom50jc/des_encoded_flip_2.npy',reps)

In [38]:
e = np.load('/external/grant/data/moco/DES/des_encoded.npy')
print(e.shape)

(1000, 2048)


In [4]:
res50 = models.resnet50
print(res50())

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [6]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, trac